# Data Exploration with BigQuery

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ayoisio/genai-on-google-cloud/blob/main/chapter-2/colabs/01_data_exploration_bigquery.ipynb)

**Estimated Time**: 10-15 minutes

**Prerequisites**: Google Cloud project with billing enabled, BigQuery API enabled

---

## Overview

Before building LLM applications, you need to understand your data. This notebook demonstrates how to:

1. **Explore datasets** using BigQuery's INFORMATION_SCHEMA
2. **Assess data quality** with profiling queries
3. **Understand data structure** for LLM readiness

These foundational skills are essential for the data readiness concepts covered in Chapter 2.

## 1. Setup & Authentication

In [ ]:
# @title Install Dependencies
!pip install --upgrade google-cloud-bigquery google-cloud-aiplatform -q

In [ ]:
# @title Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()
print("✓ Authentication successful")

In [ ]:
# @title Configure Your Project
PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Validate project ID
if PROJECT_ID == "your-project-id":
    raise ValueError("Please set your PROJECT_ID above")

print(f"✓ Project: {PROJECT_ID}")
print(f"✓ Location: {LOCATION}")

In [ ]:
# @title Initialize BigQuery Client
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID)
print(f"✓ BigQuery client initialized for project: {PROJECT_ID}")

## 2. Exploring Public Datasets

BigQuery provides access to many public datasets that are ideal for experimentation. Let's explore the Wikipedia dataset, which is commonly used for NLP and LLM applications.

In [ ]:
# @title List tables in a public dataset
DATASET_ID = "bigquery-public-data.samples"

query = f"""
SELECT 
    table_name,
    table_type,
    creation_time,
    ROUND(size_bytes / 1024 / 1024, 2) as size_mb
FROM `bigquery-public-data.samples.INFORMATION_SCHEMA.TABLES`
ORDER BY size_bytes DESC
"""

tables_df = bq_client.query(query).to_dataframe()
print(f"Found {len(tables_df)} tables in {DATASET_ID}:\n")
display(tables_df)

In [ ]:
# @title Examine table schema
TABLE_ID = "wikipedia"  # @param {type:"string"}

query = f"""
SELECT 
    column_name,
    data_type,
    is_nullable
FROM `bigquery-public-data.samples.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = '{TABLE_ID}'
ORDER BY ordinal_position
"""

schema_df = bq_client.query(query).to_dataframe()
print(f"Schema for {TABLE_ID}:\n")
display(schema_df)

In [ ]:
# @title Preview sample data
query = """
SELECT 
    title,
    SUBSTR(text, 1, 200) as text_preview,
    LENGTH(text) as text_length
FROM `bigquery-public-data.samples.wikipedia`
WHERE text IS NOT NULL
LIMIT 5
"""

sample_df = bq_client.query(query).to_dataframe()
print("Sample Wikipedia articles:\n")
display(sample_df)

## 3. Data Quality Assessment

Understanding data quality is crucial for LLM applications. Poor quality data leads to poor model outputs ("Garbage In, Garbage Out"). Let's assess key quality dimensions:

- **Completeness**: How much data is missing?
- **Consistency**: Are values within expected ranges?
- **Distribution**: What does the data look like statistically?

In [ ]:
# @title Check data completeness (null values)
query = """
SELECT
    COUNT(*) as total_rows,
    COUNTIF(title IS NULL) as null_titles,
    COUNTIF(text IS NULL) as null_text,
    COUNTIF(text IS NULL OR LENGTH(text) = 0) as empty_text,
    ROUND(COUNTIF(text IS NOT NULL AND LENGTH(text) > 0) * 100.0 / COUNT(*), 2) as completeness_pct
FROM `bigquery-public-data.samples.wikipedia`
"""

quality_df = bq_client.query(query).to_dataframe()
print("Data Completeness Analysis:\n")
display(quality_df)

In [ ]:
# @title Analyze text length distribution
query = """
SELECT
    MIN(LENGTH(text)) as min_length,
    MAX(LENGTH(text)) as max_length,
    ROUND(AVG(LENGTH(text)), 0) as avg_length,
    APPROX_QUANTILES(LENGTH(text), 4)[OFFSET(2)] as median_length,
    APPROX_QUANTILES(LENGTH(text), 100)[OFFSET(90)] as p90_length,
    APPROX_QUANTILES(LENGTH(text), 100)[OFFSET(99)] as p99_length
FROM `bigquery-public-data.samples.wikipedia`
WHERE text IS NOT NULL
"""

length_df = bq_client.query(query).to_dataframe()
print("Text Length Distribution:\n")
display(length_df)

print("\n💡 Tip: Understanding text length helps determine chunking strategies for RAG.")

In [ ]:
# @title Create a data profiling summary
query = """
WITH text_stats AS (
    SELECT
        LENGTH(text) as text_length,
        ARRAY_LENGTH(SPLIT(text, ' ')) as word_count
    FROM `bigquery-public-data.samples.wikipedia`
    WHERE text IS NOT NULL AND LENGTH(text) > 0
)
SELECT
    'Total Documents' as metric,
    CAST(COUNT(*) as STRING) as value
FROM text_stats
UNION ALL
SELECT
    'Avg Words per Document',
    CAST(ROUND(AVG(word_count), 0) as STRING)
FROM text_stats
UNION ALL
SELECT
    'Avg Characters per Document',
    CAST(ROUND(AVG(text_length), 0) as STRING)
FROM text_stats
UNION ALL
SELECT
    'Documents > 10K chars',
    CAST(COUNTIF(text_length > 10000) as STRING)
FROM text_stats
"""

profile_df = bq_client.query(query).to_dataframe()
print("📊 Data Profile Summary:\n")
display(profile_df)

## 4. Identifying LLM-Ready Data

Not all data is suitable for LLM applications. Let's identify documents that meet quality thresholds for RAG or fine-tuning.

In [ ]:
# @title Find high-quality documents for LLM use
MIN_WORDS = 100  # @param {type:"integer"}
MAX_WORDS = 5000  # @param {type:"integer"}

query = f"""
WITH document_analysis AS (
    SELECT
        title,
        text,
        LENGTH(text) as char_count,
        ARRAY_LENGTH(SPLIT(text, ' ')) as word_count
    FROM `bigquery-public-data.samples.wikipedia`
    WHERE text IS NOT NULL AND LENGTH(text) > 0
)
SELECT
    title,
    word_count,
    char_count,
    SUBSTR(text, 1, 150) as text_preview
FROM document_analysis
WHERE word_count BETWEEN {MIN_WORDS} AND {MAX_WORDS}
ORDER BY word_count DESC
LIMIT 10
"""

quality_docs_df = bq_client.query(query).to_dataframe()
print(f"📝 Documents with {MIN_WORDS}-{MAX_WORDS} words (ideal for RAG):\n")
display(quality_docs_df)

In [ ]:
# @title Summarize LLM readiness
query = f"""
WITH document_analysis AS (
    SELECT
        ARRAY_LENGTH(SPLIT(text, ' ')) as word_count
    FROM `bigquery-public-data.samples.wikipedia`
    WHERE text IS NOT NULL AND LENGTH(text) > 0
)
SELECT
    COUNT(*) as total_documents,
    COUNTIF(word_count < {MIN_WORDS}) as too_short,
    COUNTIF(word_count BETWEEN {MIN_WORDS} AND {MAX_WORDS}) as ideal_length,
    COUNTIF(word_count > {MAX_WORDS}) as too_long,
    ROUND(COUNTIF(word_count BETWEEN {MIN_WORDS} AND {MAX_WORDS}) * 100.0 / COUNT(*), 2) as ideal_pct
FROM document_analysis
"""

readiness_df = bq_client.query(query).to_dataframe()
print("🎯 LLM Readiness Summary:\n")
display(readiness_df)

ideal_pct = readiness_df['ideal_pct'].iloc[0]
print(f"\n✅ {ideal_pct}% of documents are in the ideal length range for RAG applications.")

## 5. Try It Yourself

Now it's your turn! Modify the queries below to explore different aspects of the data.

In [ ]:
# TODO: Write a query to find the top 10 longest Wikipedia articles
# Hint: Use ORDER BY with LENGTH(text) DESC

your_query = """
-- Your query here
SELECT 
    title,
    LENGTH(text) as text_length
FROM `bigquery-public-data.samples.wikipedia`
WHERE text IS NOT NULL
ORDER BY LENGTH(text) DESC
LIMIT 10
"""

result_df = bq_client.query(your_query).to_dataframe()
display(result_df)

In [ ]:
# TODO: Explore a different public dataset
# Try: bigquery-public-data.patents_view.patent

explore_query = """
SELECT 
    table_name,
    ROUND(size_bytes / 1024 / 1024 / 1024, 2) as size_gb
FROM `bigquery-public-data.patents_view.INFORMATION_SCHEMA.TABLES`
ORDER BY size_bytes DESC
LIMIT 5
"""

result_df = bq_client.query(explore_query).to_dataframe()
print("Patents dataset tables:")
display(result_df)

## Summary

In this notebook, you learned how to:

1. ✅ **Explore datasets** using BigQuery's INFORMATION_SCHEMA
2. ✅ **Assess data quality** with completeness and distribution checks
3. ✅ **Identify LLM-ready data** based on quality thresholds

### Key Takeaways

- **Data profiling** is essential before building LLM applications
- **Text length analysis** helps determine chunking strategies for RAG
- **BigQuery public datasets** provide excellent experimentation data

---

## Next Steps

Continue to the next notebook: **[02_document_processing.ipynb](02_document_processing.ipynb)** to learn how to extract structured data from unstructured documents using Document AI.